Simple multi-agent LLM system is designed using the Flan-T5-Base model, where each agent (planner, worker/answerer, summarizer) performs specific subtasks such as breaking down queries, generating responses, and combining results, with communication implemented via message passing to enable collaborative problem-solving in a lightweight, API-free, and modular setup suitable for low-resource environments.

In [1]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer


d:\Undergraduate_Ai\Sem-7\LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [3]:
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)

Device set to use cpu


#### Planner Agent

In [4]:
class PlannerAgent:
    def plan(self, question):
        prompt = f"Break the following task into 2-3 subtasks:\n{question}"
        response = pipe(prompt)[0]["generated_text"]
        return [task.strip() for task in response.split('\n') if task.strip()]

#### Answer Agent


In [5]:
class AnswererAgent:
    def answer(self, subtask, context):
        prompt = f"Answer this subtask using the given context.\nSubtask: {subtask}\nContext: {context}\nAnswer:"
        return pipe(prompt)[0]["generated_text"]


#### Summarizer Agent

In [6]:
class SummarizerAgent:
    def summarize(self, responses):
        prompt = f"Summarize the following responses into a concise final answer:\n{' '.join(responses)}"
        return pipe(prompt)[0]["generated_text"]

#### Simulater context source


In [7]:
def get_context(subtask):
    contexts = {
        "Find recent data on climate change effects.": "Global temperatures have risen by ~1.2°C since pre-industrial times...",
        "Analyze how temperature changes affect crop yields.": "Higher temperatures reduce wheat and rice yields due to heat stress...",
        "Summarize agricultural adaptation strategies.": "Farmers are adopting drought-resistant crops, precision irrigation, and agroforestry..."
    }
    return contexts.get(subtask, "No context available.")



#### Run mullti-agent system

In [9]:
if __name__ == "__main__":
    planner = PlannerAgent()
    answerer = AnswererAgent()
    summarizer = SummarizerAgent()

    user_query = "Explain the impact of climate change on global agriculture."

    print("Planning Subtasks...")
    subtasks = planner.plan(user_query)
    print("Subtasks:", subtasks)

    print("\nSolving Subtasks...")
    answers = []
    for subtask in subtasks:
        context = get_context(subtask)
        answer = answerer.answer(subtask, context)
        answers.append(answer)
        print(f"\n✅ {subtask}\n{answer}")

    print("\nFinal Summary:")
    summary = summarizer.summarize(answers)
    print(summary)

Planning Subtasks...
Subtasks: ['The impact of climate change on global agriculture.']

Solving Subtasks...

✅ The impact of climate change on global agriculture.
There is no evidence that climate change has a negative impact on global agriculture.

Final Summary:
It is not clear whether climate change has a negative impact on global agriculture.
